# Unit 3: Neural Network

# Neural network
Dữ liệu từ input layer sẽ được tính feedforward qua các hidden layer để ra được output layer.
![](https://i1.wp.com/nttuan8.com/wp-content/uploads/2019/03/fw.png?fit=1024%2C98&ssl=1)

Sau khi có giá trị dữ đoán $\hat{y}$ thì mình sẽ tạo loss function và dùng thuật toán backpropagation để tính đạo hàm của loss với các tham số W, b, rồi dùng thuật toán gradient descent để cập nhật hệ số và tối ưu loss function.

## Linear regression

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
# nn.Linear(số_node_input, số node_output) trong layer đó
linear_model = nn.Linear(1, 1)

In [3]:
print(list(linear_model.parameters()))
print(f"Total parameters: {len(list(linear_model.parameters()))}")

# [Parameter containing: tensor([[14.9464]], requires_grad=True), Parameter containing: tensor([1.1261], requires_grad=True)]

[Parameter containing:
tensor([[-0.9829]], requires_grad=True), Parameter containing:
tensor([-0.5027], requires_grad=True)]
Total parameters: 2


Trong mỗi layer sẽ gồm 2 Parameters là `W` và `b` tương ứng. Mình thấy là thuộc tính `requires_grad` của các Parameter đều là `True` để có thể tính `backward` loss và dùng gradient descent.

In [4]:
loss_fn = nn.MSELoss()

Trong bài trước mình phải tự tính `L.backward()` rồi thực hiện `gradient descent` bằng tay, tuy nhiên là Pytorch hỗ trợ các `optimizer` trong `nn.optim` để giúp mọi người thực hiện `gradient descent` luôn. Mình cần truyền cho `optimizer` biết là: mình muốn thực hiện `gradient descent` với những tham số nào (thường là tất cả các tham số trong `model`) cũng như `learning rate` là bao nhiêu.

In [5]:
optimizer = optim.SGD(linear_model.parameters(), lr=0.00004)

Training loop
```python
for epoch in tqdm(range(1, n_epochs+1)):
    y_hat = model(X)
    loss = loss_fn(y_hat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
```

Mình tính giá trị dự đoán, sau đó tính `loss`, rồi gọi `loss.backward()` để tính đạo hàm ngược lại cho các `W`, `b`. *Tuy nhiên* vì mỗi khi gọi `loss.backward()` thì đạo hàm ở các tham số sẽ cộng dồn lại, nên mình cần gọi zero_grad để gán đạo hàm ở các tham số bằng 0 trước khi gọi hàm backward(). Cuối cùng gọi `optimizer.step()` để thực hiện `update gradient descent` trên các tham số trong `optimizer`.

Khi mình cho dữ liệu vào `model` để học, thì thay vì cho học từng dữ liệu một, mình sẽ cho học nhiều điểm dữ liệu một lúc theo `batch`, dữ liệu mình truyền vào sẽ có kích thước `(batch_size, 1)` hay trong trường hợp tổng quát với NN là `(batch_size, num_features)`, viết tắt `(N*d)`.

```python
x = torch.tensor(data[:,0], dtype=torch.float32) # x.shape (30)
# Thêm chiều vào vị trí tương ứng, ví dụ (30) -> (30, 1).
x = x.unsqueeze(1) # x.shape (30, 1)
```

Cuối cùng xong mình sẽ tìm được model, các điểm xanh dương là dữ liệu của mình. Còn đường đỏ là model dự đoán.
![](https://i0.wp.com/nttuan8.com/wp-content/uploads/2021/03/image-38.png?w=604&ssl=1)

### Neural Network

Mình sẽ xây dựng mạng CNN cho bài toán phân loại ảnh MNIST.

Dữ liệu ảnh trong dataset MNIST là ảnh xám và có kích thước 28*28. Bài toán input là 1 ảnh, output xem ảnh đấy là số mấy 0->9.
![](https://i2.wp.com/nttuan8.com/wp-content/uploads/2019/04/model.png?fit=1024%2C386&ssl=1)

In [6]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()
        # input 1 channel, output 32 channel, kernel size 3*3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        
        self.fc1 = nn.Linear(14*14*32, 128)  # 14*14 from image dimension
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

        
    # def forward(self, x):
    #     # Max pooling 2*2
    #     x = F.relu(self.conv1(x))

    #     x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    #     # Flatten về dạng vector để input vào mạng NN
    #     x = torch.flatten(x)
    #     x = F.relu(self.fc1(x))
    #     x = self.fc2(x)
    #     return x


In [7]:
net = CNN()
print(net)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=6272, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [8]:
params = list(net.parameters())
print("total weights: %d" %(sum(p.numel() for p in params))) # 8, do có 4 layers, mỗi layer có 2 Parameter là W và b

total weights: 813802


In [9]:
# shape: batch_size * depth * height * weight (N*C*H*W)
input = torch.randn(1, 1, 28, 28)
out = net(input)
print(out)

tensor([-0.0468, -0.0544,  0.0390,  0.0508, -0.0747,  0.0463, -0.1495, -0.0909,
        -0.1038,  0.0506], grad_fn=<AddBackward0>)


## Neural network Tips

### Hooks functions
- Forward hook
- Backward hook

In [12]:
net = CNN()
def print_info(self, input, output):
    # input is a tuple of inputs
    # Output is a tensor, output value is in output.data
    print('Inside' + self.__class__.__name__ + 'forward')
    
    print('')
    print('input: ', type(input), ', len: ', len(input))
    print('input[0]: ', type(input[0]), ', shape: ', input[0].shape)
    print('output: ', type(output), ', len: ', len(output), output.data.shape)

# Register a function to forward hook
net.conv1.register_forward_hook(print_info)
net.conv2.register_forward_hook(print_info)

# Forward hook function will be called when model computing forward through conv1
out = net(input)

InsideConv2dforward

input:  <class 'tuple'> , len:  1
input[0]:  <class 'torch.Tensor'> , shape:  torch.Size([1, 1, 28, 28])
output:  <class 'torch.Tensor'> , len:  1 torch.Size([1, 32, 28, 28])
InsideConv2dforward

input:  <class 'tuple'> , len:  1
input[0]:  <class 'torch.Tensor'> , shape:  torch.Size([1, 32, 28, 28])
output:  <class 'torch.Tensor'> , len:  1 torch.Size([1, 32, 28, 28])


Tương tự, mình cũng có thể register 1 hàm thành backward hook.

In [15]:
net = CNN()

def print_backward_info(self, grad_input, grad_output):
    # Grad_input and grad_output are tuples
    print('Inside ' + self.__class__.__name__ + ' backward')
    
    print('grad_input: ', type(grad_input), ', len: ', len(grad_input))
    print('grad_output: ', type(grad_output), ', len: ', len(grad_output))
    print('grad_output[0]: ', type(grad_output[0]), ', size: ', grad_output[0].shape)
    print('')

# Register a function to backward hook
net.conv1.register_backward_hook(print_backward_info)
net.conv2.register_backward_hook(print_backward_info)

out = net(input)
target = torch.rand(10)
err = loss_fn(out, target)


# Backward hook will be called when loss backward through layer conv1 and conv2
err.backward()




Inside Conv2d backward
grad_input:  <class 'tuple'> , len:  3
grad_output:  <class 'tuple'> , len:  1
grad_output[0]:  <class 'torch.Tensor'> , size:  torch.Size([1, 32, 28, 28])

Inside Conv2d backward
grad_input:  <class 'tuple'> , len:  3
grad_output:  <class 'tuple'> , len:  1
grad_output[0]:  <class 'torch.Tensor'> , size:  torch.Size([1, 32, 28, 28])



/home/toan/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1113: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


### Model __call__ vs forward
Về cơ bản thì khi mọi người build xong model thì kết quả của việc mọi người dùng __call__ (net(input)) hay dùng forward là như nhau.

In [16]:
input = torch.randn(1, 1, 28, 28)
out_call = net(input)
out_forward = net.forward(input)

out_call == out_forward # True

tensor([True, True, True, True, True, True, True, True, True, True])

Tuy nhiên __call__ ngoài việc thực hiện forward sẽ gọi các hook nữa, thế nên khi dùng mọi người nên dùng __call__ và tránh dùng forward.

In [17]:
def __call__(self, *input, **kwargs):
  for hook in self._forward_pre_hooks.values():
    hook(self, input)
  
  result = self.forward(*input, **kwargs)
  
  for hook in self._forward_hooks.values():
    hook(self, input, result)
    # TODO

  return result

### nn.relu vs F.relu

Mọi người để ý ở trên, để áp dụng ReLU activation mình dùng `F.relu(...)`. Tuy nhiên trong module torch.nn cũng có `torch.nn.ReLU(...)`.

Về cơ bản thì 2 hàm thực hiện chức năng giống nhau, áp dụng ReLU activation function. Tuy nhiên điểm khác nhau lớn nhất là: F.relu như một hàm tính ReLU bình thường, tuy nhiên nn.ReLU tạo ra 1 nn.Module giống như các layer Linear, Conv2d. Do đó nn.ReLU có thể thêm vào nn.Sequential, cũng như register các hàm hook.

Những hàm để xây dựng model như Linear, Conv2d, ReLU, max_pool2d đều có cả 2 dạng nn và F. Nó phụ thuộc vào phong cách code và xây dựng model của mỗi người. Mình thì thường dùng nn với những hàm có tham số, còn không có tham số như ReLU, max_pool2d thì dùng functional (F).